In [4]:
import pandas as pd

In [71]:
with open('topics.csv') as csv:
    with open('newtopics.csv','w') as result:
        for line in csv:
            columns = line.split(',')
            num = len(line.split(','))
            columns[1] = ' + '.join(columns[1:])
            new_column = columns[0]+','+columns[1].replace("\n", "")+'\n'
            result.write(new_column.lower())

In [86]:
df = pd.read_csv('newtopics.csv', header= 0, names=['序号', '关键词'], index_col=0)
df = df[df['关键词'].str.contains('complexity')]
df2 = pd.read_csv('papers_changed.csv', index_col=0)

In [95]:
list_url = [i for i in df.join(df2['link'], how='inner')['link']]
list_url

['https://doi.org/10.1093/applin/amy064',
 'https://doi.org/10.1093/applin/amz005',
 'https://doi.org/10.1093/applin/amz034',
 'https://doi.org/10.1093/applin/amx001',
 'https://doi.org/10.1093/applin/amx031']

从sci-hub爬取相关文章并保存为pdf。

In [109]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
ua = UserAgent()
import random, time

In [110]:
domain_scihub = 'https://sci-hub.tf/'

In [ ]:
for doi in list_url:
    filename = doi[-6:]+'.pdf'
    sci_url = domain_scihub + doi
    headers = {'User-Agent': ua.random}
    bs = BeautifulSoup(requests.get(sci_url, headers=headers).content, "lxml")
    # 下面这一步是关键：
    pdf_link = bs.find('div', id = 'buttons').find_all('a')[1].attrs.get("onclick").replace('location.href=','')[1:-1]
    if 'http:' in pdf_link:
        continue
    else:
        pdf_link = 'http:' + pdf_link
    response = requests.get(pdf_link, headers=headers)
    with open('complexity '+ filename, 'wb') as f:
        f.write(response.content)
    print('Downloaded: '+ filename)
    time.sleep(random.random()*10)  

Downloaded: amy064.pdf
Downloaded: amz005.pdf
